In [ ]:
import geemap.eefolium as emap
import ee

In [ ]:
#initialize GEE in Jupyter Notebook session
ee.Authenticate()
ee.Initialize()

In [ ]:
#map object
Map = emap.Map()

#get data and add layers to map
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

#add image to map
Map.centerObject(image, 8)
vizParams = {'bands': ['B5', 'B4', 'B3'],'min': 5000, 'max': 15000, 'gamma': 1.3}
Map.addLayer(image, vizParams, 'Landsat 8 original image')

#add other layer to map
counties = ee.FeatureCollection('TIGER/2016/Counties')
Map.addLayer(ee.Image().paint(counties, 0, 2), {}, 'counties')

#display data
Map.addLayerControl()
Map

In [ ]:
def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B4']))

def cloudMask(image):
    clouds = ee.Algorithms.Landsat.simpleCloudScore(image).select(['cloud'])
    return image.updateMask(clouds.lt(10))

# Load a Landsat collection, map the NDVI and cloud masking functions over it.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(ee.Geometry.Point([-122.262, 37.8719])) \
    .filterDate('2014-03-01', '2014-05-31') \
    .map(addNDVI) \
    .map(cloudMask)

centroid = collection.first().geometry().centroid().getInfo()['coordinates']

Map = emap.Map(center=centroid, zoom=7)

# Reduce the collection to the mean of each pixel and display.
meanImage = collection.reduce(ee.Reducer.mean())
vizParams = {'bands': ['B5_mean', 'B4_mean', 'B3_mean'], 'min': 0, 'max': 0.5}
Map.setCenter(-122.262, 37.8719, 10)
Map.addLayer(meanImage, vizParams, 'mean')

# Load a region in which to compute the mean and display it.
counties = ee.FeatureCollection('TIGER/2016/Counties')
santaClara = ee.Feature(counties.filter(
    ee.Filter.eq('NAME', 'Santa Clara')).first())
Map.addLayer(ee.Image().paint(santaClara, 0, 2), {
             'palette': 'yellow'}, 'Santa Clara')

# Get the mean of NDVI in the region.
mean = meanImage.select(['nd_mean']).reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': santaClara.geometry(),
    'scale': 30
})

Map.addLayer(meanImage.clip(santaClara.geometry()), vizParams, 'mean2')


# # Print mean NDVI for the region.
# print('Santa Clara spring mean NDVI:', mean.get('nd_mean').getInfo())

In [ ]:
#display data
Map.addLayerControl()
Map

In [ ]:
Map = emap.Map(center=[40,-100], zoom=4)

# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
})

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.cart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Display the inputs and the results.
Map.centerObject(points, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
Map.addLayer(classified,
             {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')

In [ ]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map